In [16]:
import pandas as pd
import numpy as np

In [18]:
data = pd.read_csv('cs_bisnode_panel.csv')

In [20]:
# Filter the data for the industry of interest (ind2 == 26)
industry_filtered = data[data['ind2'] == 26]

# Convert 'begin' and 'end' columns to datetime to extract the year
industry_filtered['begin_year'] = pd.to_datetime(industry_filtered['begin']).dt.year
industry_filtered['end_year'] = pd.to_datetime(industry_filtered['end']).dt.year

# Filter for SMEs based on sales in 2014 (between 1000 EUR and 10 million EUR)
sme_filtered = industry_filtered[(industry_filtered['begin_year'] == 2014) & 
                                 (industry_filtered['sales'] >= 1000) & 
                                 (industry_filtered['sales'] <= 10000000)]

# Display the shape of the filtered dataset and the first few rows to verify the filtering
filtered_shape = sme_filtered.shape
first_rows_filtered = sme_filtered.head()

filtered_shape, first_rows_filtered


C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1743481007.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_filtered['begin_year'] = pd.to_datetime(industry_filtered['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1743481007.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_filtered['end_year'] = pd.to_datetime(industry_filtered['end']).dt.year


((1037, 50),
          comp_id       begin         end  COGS        amort    curr_assets  \
 969    6538183.0  2014-01-01  2014-12-31   NaN  1018.518494   17022.222656   
 1128   8416055.0  2014-01-01  2014-12-31   NaN   425.925934  105740.742188   
 1467  12428378.0  2014-01-01  2014-12-31   NaN     0.000000       0.000000   
 1706  17776540.0  2014-01-01  2014-12-31   NaN    48.148148  142296.296875   
 1735  18626760.0  2014-01-01  2014-12-31   NaN  5755.555664   95185.187500   
 
           curr_liab  extra_exp  extra_inc  extra_profit_loss  ...  nace_main  \
 969     3040.740723        0.0        0.0                0.0  ...     2630.0   
 1128    6918.518555        0.0        0.0                0.0  ...     2651.0   
 1467    9277.777344        0.0        0.0                0.0  ...     2660.0   
 1706  164237.031250        0.0        0.0                0.0  ...     2620.0   
 1735   45766.667969        0.0        0.0                0.0  ...     2670.0   
 
       ind2  ind  urban

In [21]:
# Create a list of unique comp_id from the filtered data
unique_comp_ids = sme_filtered['comp_id'].unique()

# Initialize a dictionary to store the default status of each company
default_status = {}

# Loop through each company ID to check if it exists in 2015
for comp_id in unique_comp_ids:
    # Filter data for this company
    comp_data = data[data['comp_id'] == comp_id]
    
    # Convert 'begin' and 'end' columns to datetime to extract the year for this step
    comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
    
    # Check if there is any record for 2015
    exists_in_2015 = comp_data['begin_year'].isin([2015]).any()
    
    # Mark as default (1) if it does not exist in 2015, else 0
    default_status[comp_id] = 0 if exists_in_2015 else 1

# Convert the default_status dictionary to a DataFrame
default_status_df = pd.DataFrame(list(default_status.items()), columns=['comp_id', 'defaulted'])

# Merge this default status back to the filtered dataset
sme_filtered_with_default = sme_filtered.merge(default_status_df, on='comp_id', how='left')



C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

...

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_data['begin_year'] = pd.to_datetime(comp_data['begin']).dt.year
C:\Users\maiph\AppData\Local\Temp\ipykernel_19804\1796235787.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [22]:
# Calculate the number of defaulted and non-defaulted firms
defaulted_count = sme_filtered_with_default['defaulted'].sum()
non_defaulted_count = sme_filtered_with_default.shape[0] - defaulted_count

# Calculate average, minimum, and maximum sales
average_sales = sme_filtered_with_default['sales'].mean()
min_sales = sme_filtered_with_default['sales'].min()
max_sales = sme_filtered_with_default['sales'].max()

# Compile the statistics into a dictionary for easy presentation
statistics = {
    'Number of Defaulted Firms': defaulted_count,
    'Number of Non-Defaulted Firms': non_defaulted_count,
    'Average Sales (in million EUR)': average_sales / 1e6,  # Convert to million EUR
    'Minimum Sales (in million EUR)': min_sales / 1e6,      # Convert to million EUR
    'Maximum Sales (in million EUR)': max_sales / 1e6       # Convert to million EUR
}

statistics


{'Number of Defaulted Firms': 28,
 'Number of Non-Defaulted Firms': 1009,
 'Average Sales (in million EUR)': 0.49020221792682,
 'Minimum Sales (in million EUR)': 0.001070370361328125,
 'Maximum Sales (in million EUR)': 9.576485}

In [13]:
# Selecting important features for modeling
features = ['comp_id', 'sales', 'fixed_assets', 'curr_assets', 'curr_liab', 'profit_loss_year', 'defaulted']

# Creating a new DataFrame with selected features
model_data = sme_filtered_with_default[features]

# Display the first few rows of the model data to verify the selection
model_data.head()


,comp_id,sales,fixed_assets,curr_assets,curr_liab,profit_loss_year,defaulted
0,6538183.0,49688.890625,3359.259277,17022.222656,3040.740723,4144.444336,0
1,8416055.0,42737.035156,7618.518555,105740.742188,6918.518555,-3088.888916,0
2,12428378.0,3992.592529,0.000000,0.000000,9277.777344,-10662.962891,0
3,17776540.0,212225.921875,180137.031250,142296.296875,164237.031250,3625.926025,0
4,18626760.0,168248.140625,22818.517578,95185.187500,45766.667969,6266.666504,0


In [23]:
# Fill missing values with the mean of the column
model_data_filled = model_data.fillna(model_data.mean())

# Verify if there are any missing values left
missing_values_after_filling = model_data_filled.isnull().sum()

# Create a new feature - debt to assets ratio
model_data_filled['debt_to_assets_ratio'] = model_data_filled['curr_liab'] / model_data_filled['curr_assets']


In [24]:
# Replace inf values with NaN
model_data_filled['debt_to_assets_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with the mean of the column
model_data_filled['debt_to_assets_ratio'].fillna(model_data_filled['debt_to_assets_ratio'].mean(), inplace=True)

# Verify the replacement and filling
final_check = model_data_filled.isnull().sum()

